<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/Cadenas%20b%C3%A1sicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Cadenas en Langchain**

Las cadenas en LangChain representan una secuencia de operaciones que procesan datos de entrada a través de uno o más modelos de lenguaje. Cada operación de una cadena puede considerarse como un paso de un flujo de trabajo, donde el resultado de un paso sirve como entrada para el siguiente. Estas cadenas pueden ser simples o complejas, según los requisitos del proyecto.

## **Nuevas cadenas con LangChain Expression Language (LCEL)**
---

En octubre de 2023, LangChain introdujo una nueva forma de trabajar con cadenas, más eficiente y simple, utilizando el operador | (pipe).  

Esto permite encadenar elementos de manera intuitiva, con soporte para stream, batch y operaciones asíncronas.  

Además, con LCEL (LangChain Expression Language), las cadenas se pueden definir de forma declarativa y funcional, eliminando la necesidad de clases complejas.   

Aunque las cadenas "antiguas" siguen funcionando, es recomendable adoptar esta nueva sintaxis.

### Ventajas del nuevo sistema:

- **Alineación con el framework:** La nueva sintaxis es el estándar recomendado para futuros proyectos.
- **Interface unificada:** Simplifica la integración de múltiples métodos.
- **Facilidad de composición:** Permite estructurar cadenas de forma secuencial, paralela o con fallbacks.
- **Código más limpio:** Menos líneas para lograr los mismos resultados.

En LCEL **(LangChain Expression Language**), no existen como tal tipos de cadenas predefinidos como "secuenciales", "condicionales" o "paralelas", pero puedes implementar estos patrones de flujo de trabajo utilizando la sintaxis de LCEL y algunas funciones adicionales.

En este cuaderno nos vamos a ocupar solo del patron secuencial. En proximos cuadernos veremos el resto de patrones de diseño.

## **Configuración del entorno del cuaderno**
---

Configuramos el entorno de trabajo para utilizar LangChain con distintos modelos de lenguaje (LLMs).  
1. Obtenemos las claves API para acceder a los servicios de OpenAI, Groq, Google y Hugging Face.

2. Instalamos la librería LangChain y las integraciones necesarias para cada uno de estos proveedores.

3. Importamos las clases específicas de LangChain que permiten crear plantillas de prompts e interactuar con los diferentes modelos de lenguaje, dejándolo todo listo para empezar a desarrollar aplicaciones basadas en LLMs. (Este codigo se explico con detalle en el primer cuaderno)

Comenta (#) las librerias y modelos que no desees usar.

In [ ]:
# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')

# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
!pip install langchain -qU  # Instalar la librería principal de LangChain.

# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
!pip install langchain-openai -qU
!pip install langchain-groq -qU
!pip install langchain-google-genai -qU
!pip install langchain-huggingface -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate


# Importamos las clases necesarias para trabajar con cadenas
from langchain.chains import LLMChain

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint

# Importamos la libreria para formatear mejor la salida
from IPython.display import Markdown, display

## **Ejemplo 1: Prompt + LLM**

---
Posiblemente esta es la cadena mas simple que podamos crear.  
En este caso la cadena conecta un prompt con un modelo.

Se usa | para conectar el prompt con el modelo.
Luego la cadena se invoca pasandole el PrompTemplate.

Observa que esto es basicamente lo que hemos venido haciendo hasta ahora sin cadenas.

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)

# Definimos el prompt template de una de esta formas
prompt = PromptTemplate(template="Traduce esto al {idioma}: {texto}", input_variables=["idioma","texto"])
prompt = PromptTemplate.from_template("Traduce esto al {idioma}: {texto}")

# Usamos LCEL para crear la cadena
chain = prompt | llm

# Ejecutamos la cadena
# El input del metodo .invoke de una cadena debe ser un unico
# asi que puede hacerse asi,
#     respuesta = chain.invoke("Hola, ¿cómo estás?")
# pero si hay mas de un parametro se espera un diccionario

respuesta = chain.invoke({"idioma":"Francés", "texto":"Hola, ¿cómo estás?"})
display(Markdown(respuesta.content))

Observa que esto es lo que hemos hecho hasta ahora sin cadenas: Invocar un LLM pasandole un PromptTemplate formateado...

Aunque no hay gran diferencia y este enfoque puede ser más simple para tareas sencillas, carece de la flexibilidad y la extensibilidad que ofrecen las cadenas como se aprecia en cuanto la complejidad escala.

Ademas LCEL y la nuevas cadenas proporcionan un nivel de abstracción mayor tanto del prompt como del proceso en general.


In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)

# Creamos el prompt y lo formateamos
prompt_template = PromptTemplate.from_template("Traduce esto al {idioma}: {texto}")
prompt_formateado = prompt_template.format(idioma="Francés", texto="Hola, ¿cómo estás?")

# Invocamos el LLM directamente
respuesta = llm.invoke(prompt_formateado)

display(Markdown(respuesta.content))

## **Ejemplo 2: Prompt + LLM + funcion personalizada de transformación de la salida**
---

En este caso la salida del LLM se procesa con una funcion sencilla para pasarla a mayusculas. (pero puede ser tan complicada como se necesite)

Observa que la funcion de transformacion recibe la salida del LLM que es un objeto AIMessage. Asi que debe acceder al .content de este (que es realmente el string con la respuesta) para poder operar.

Vamos a usar en adelante display de IPython (en lugar de print) para imprimir el markdown resultante y que se vea menjor en pantalla.

In [ ]:
# Definimos el PromptTemplate
prompt_template = PromptTemplate.from_template("Responde como si fueras un experto en {tema}: {pregunta}")

# Recuerda que esto es quivalente, con el constructor de clase
# prompt_template = PromptTemplate(
#     template="Responde como si fueras un experto en {tema}: {pregunta}",
#     input_variables=["tema", "pregunta"]
# )


# Definimos el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)

# Definimos una función de transformación
def transformar_texto(output):
    return output.content.upper()

# Creamos la cadena con LCEL
chain = prompt_template | llm | transformar_texto

# Ejecutamos la cadena
respuesta = chain.invoke({"tema": "Machine Learning", "pregunta": "¿Qué es el overfitting?"})
display(Markdown(respuesta))

## **Ejemplo 3: Prompt + LLM + OutputParser + funcion personalizada de transformación de la salida**
---

Este ejemplo mejora la modularidad reutilizando la cadena en un bucle y usando un OutputParser.

En LangChain, los **parsers de salida** estructuran la salida del LLM en formatos específicos (string, JSON, lista, etc.).

**Tipos comunes:**

- **StrOutputParser**: Convierte a string (usado aquí).
    
- **JsonOutputParser**: Convierte a JSON.
    
- **CommaSeparatedListOutputParser**: Crea listas separadas por comas.
    
- **StructuredOutputParser**: Define esquemas de salida personalizados.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# 1. Definimos el PromptTemplate
prompt_template = PromptTemplate.from_template("Responde como si fueras un experto en {tema}: {pregunta}")

# 2. Definimos el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)

# 3. Definimos un OutputParser
# StrOutputParser convierte la salida del LLM en un string.
output_parser = StrOutputParser()

# 4. Definimos una función de transformación
# Esta función toma la salida del OutPutParser (str) y la convierte a mayúsculas.
def transformar_texto(output):
    return output.upper()

# 5. Creamos la cadena con LCEL
chain = prompt_template | llm | output_parser | transformar_texto

# 6. Ejecutamos la cadena con diferentes entradas en un bucle
# Observa cómo la cadena puede reutilizarse para diferentes preguntas.
preguntas = [
    {"tema": "Machine Learning", "pregunta": "¿Qué es el overfitting?"},
    {"tema": "Historia", "pregunta": "¿Quién fue Napoleón Bonaparte?"},
    {"tema": "Programación", "pregunta": "¿Qué es Python?"}
]

for pregunta in preguntas:
    respuesta = chain.invoke(pregunta)
    display(Markdown(f"**Pregunta:** {pregunta['pregunta']}"))
    display(Markdown(f"**Respuesta:** {respuesta}"))
    print("---")  # Separador entre preguntas

## **Ejemplo 4: Prompt + LLM + OutputParser + Guardar en fichero**
---
Crea una cadena que genera una lista de elementos y la guarda en un archivo:

- **Prompt:** Define un ChatPromptTemplate que instruye al sistema a crear listas sin numerar, en una columna y sin comentarios.
    
- **LLM:** Usa el modelo gpt-4o-mini de OpenAI para generar la respuesta.
    
- **OutputParser:** Formatea la respuesta como texto plano con StrOutputParser.
    
- **Función Personalizada:** La función guardar\_en\_archivo guarda la respuesta en "respuesta.txt" e imprime la respuesta en la consola.  

Esta vez, para una mayor correcion, aunque no es necesario, vamos a usar ChatPromptTemplate

In [ ]:
# Definimos el PromptTemplate
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente conciso y directo. Proporcionas las listas que se te piden y solo la lista sin numerar y en una columna. No añadas ningun tipo de comentario"),
    ("human", "Crea una lista de {numero_elementos} {tipo_elementos}"),
])

# Definimos el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)

# Definimos un OutputParser
output_parser = StrOutputParser()

# Definimos una función para guardar en archivo
def guardar_en_archivo(output):
    with open("respuesta.txt", "w") as f:
        f.write(output)
        print(output)
    return "Respuesta guardada en 'respuesta.txt'"

# Creamos la cadena con LCEL
chain = prompt_template | llm | output_parser | guardar_en_archivo

# Ejecutamos la cadena
respuesta = chain.invoke({"numero_elementos": 10, "tipo_elementos": "Adbervios de modo que no acaben en -mente"})
display(Markdown(respuesta))

## **Ejemplo 5: Prompt + LLM + JsonOutputParser**
---

Esta vez genera una respuesta JSON.

SimpleJsonOutputParser transforma la salida del modelo en un objeto JSON válido.


In [ ]:
from langchain.output_parsers.json import SimpleJsonOutputParser

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente conciso y directo experto en {tema}. Devuelve el resultado en formato JSON con las claves 'respuesta' y 'explicacion'. No añadas ningun tipo de comentario"),
    ("human", "{pregunta}"),
])

# Definimos el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)


json_parser = SimpleJsonOutputParser()


# Creamos la cadena con LCEL
chain = prompt_template | llm | json_parser

# Ejecutamos la cadena
result = chain.invoke({"tema": "Python", "pregunta": "¿Qué es el Streamlit?"})
print(result)

####**👉🏻¿ si el modelo ya da la respuesta en JSON (se le ha indicado en el prompt) porque se se utiliza json_parser = SimpleJsonOutputParser() para contruir la salida ?**


La respuesta tiene que ver con cómo funcionan los modelos de lenguaje y cómo se manejan sus salidas en LangChain. Vamos a desglosarlo:

#### A\. **El modelo devuelve una cadena de texto**

Aunque le indiques al modelo que genere una respuesta en formato JSON, la salida del modelo sigue siendo una **cadena de texto** (un `string`). Por ejemplo, el modelo podría devolver algo como esto:

```
'{"respuesta": "Streamlit es una biblioteca de Python para crear aplicaciones web interactivas.",
 "explicacion": "Permite a los desarrolladores crear interfaces de usuario rápidamente utilizando
 scripts de Python."}'
```

Esta cadena de texto **no es un objeto JSON** en sí misma, sino una representación textual de un JSON. Para poder trabajar con ella en Python (por ejemplo, acceder a las claves `respuesta` y `explicacion`), necesitas convertirla en un diccionario de Python.

#### B\. **¿Qué hace `SimpleJsonOutputParser`?**

`SimpleJsonOutputParser` se encarga de:

1. **Tomar la cadena de texto** que el modelo ha generado.
    
2. **Convertirla en un objeto JSON** (por ejemplo, un diccionario o una lista) utilizando `json.loads()` internamente.
    
3. **Devolver el objeto JSON** para que puedas manipularlo fácilmente en Python.

En otras palabras, **`SimpleJsonOutputParser` convierte la cadena JSON en un diccionario de Python**.


#### C\. **¿Por qué no se puede usar directamente la salida del modelo?**

Ya esta dicho. Si no usas `SimpleJsonOutputParser` (o algo similar), la salida del modelo sería una cadena de texto, no un diccionario. Por ejemplo:

```
output \= '{"respuesta": "Streamlit es una biblioteca de Python para crear aplicaciones web
interactivas.", "explicacion": "Permite a los desarrolladores crear interfaces de usuario rápidamente
utilizando scripts de Python."}'
```

Si intentas acceder a `output["respuesta"]`, obtendrás un error, porque `output` **es una cadena, no un diccionario**. Necesitas parsearla primero.


## **Ejemplo 6: Prompt + LLM + OutputParser + funcion personalizada**
---

Vamos a construir una cadena que incluya una funcion personalizada que destaque las palabras de mas de 6 letras del resultado.


In [ ]:
from langchain.schema.output_parser import StrOutputParser

# Definimos el prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Eres un traductor profesional."),
    ("human", "Traduce el siguiente texto al inglés: {texto}"),
])

# Definimos el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)

# Función personalizada para resaltar palabras clave
def resaltar_palabras_clave(output):
    # Dividimos el texto en palabras
    palabras = output.split()
    # Resaltamos las palabras clave (en este caso, palabras con más de 6 letras)
    texto_resaltado = []
    for palabra in palabras:
        if len(palabra) > 6:
            texto_resaltado.append(f"_{palabra}_")  # Resaltamos con asteriscos
        else:
            texto_resaltado.append(palabra)
    return " ".join(texto_resaltado)

# Creamos la cadena con LCEL: prompt | llm | función personalizada
chain = prompt_template | llm | StrOutputParser() | resaltar_palabras_clave

# 5. Ejecutamos la cadena
texto_original = "La inteligencia artificial está transformando la industria tecnológica."
respuesta = chain.invoke({"texto": texto_original})
display(Markdown(respuesta))

StrOutputParser se encarga de extraer el contenido de texto del objeto AIMessage generado por el modelo. Esto elimina la necesidad de acceder manualmente a output.content en la función personalizada.

## **Ejemplo 7: Cadena con dos LLMs (Generación y Resumen)**
---

Vamos a crear una cadena donde un LLM genera contenido y otro LLM resumen ese contenido.

In [ ]:
# Primer LLM: Genera contenido
llm1 = ChatOpenAI(model="gpt-4o-mini",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)

generate_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Escribe un artículo detallado sobre {topic}."
)

# Segundo LLM: Resumen del contenido generado
llm2 = ChatGoogleGenerativeAI(
    model="gemini-pro",
    api_key=GOOGLE_API_KEY,
    temperature=0.7)

summarize_prompt = PromptTemplate(
    input_variables=["article"],
    template="Resumen el siguiente artículo:\n{article}"
)

# Cadena completa usando LCEL
chain = generate_prompt | llm1 | summarize_prompt | llm2

# Ejecutar la cadena
salida = chain.invoke({"topic": "inteligencia artificial"})
display(Markdown(salida.content))

### Reto: Intenta que se visualizen ambos textos. El articulo completo y el resultado

In [ ]:
#Crea aqui tu solucioón

### Solución

## **Ejemplo 8: Integración con API Externa (Wikipedia)**
---

Podemos alimentar al modelo con el resultado de una llamada a una API externa.  
Este ejemplo creamos una cadena que usa la API de Wikipedia para obtener un resumen de un artículo utilizando ChatPromptTemplate.

Observa que en este caso la llamada a la API de Wikipedia no esta integrada en la cadena. Esto no del del todo correcto, lo hacemos asi para mantener esta vez el codigo más sencillo.  
Ademas LangChain proporciona conexiones con diferentes fuentes de datos (entre ellos Wikipedia) y trataremos esto mas adelante.

In [ ]:
import requests
from langchain.schema.output_parser import StrOutputParser

# Función para obtener datos de la API de Wikipedia
def fetch_wikipedia_summary(titulo_articulo):
    url = f"https://es.wikipedia.org/api/rest_v1/page/summary/{titulo_articulo}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json().get("extract", "No se encontró un resumen para este artículo.")
    else:
        return "Error al obtener el artículo de Wikipedia."

# LLM: Genera un resumen basado en los datos de Wikipedia
llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)

# Prompt para generar un resumen
resumen_prompt = ChatPromptTemplate.from_template(
    "Basado en la siguiente información de Wikipedia, escribe un resumen conciso en forma de viñetas o bullets points de entre 3 y 5 frases sobre:\n{info}"
)

# Cadena completa usando LCEL
chain = resumen_prompt | llm | StrOutputParser()

# Obtener el resumen de un artículo de Wikipedia
articulo = "LLM"  # Cambia esto por el título del artículo que desees
wikipedia_info = fetch_wikipedia_summary(articulo)

# Ejecutar la cadena
resumen = chain.invoke({"info": wikipedia_info})
display(Markdown(resumen))

## **Ejemplo 9: Procesamiento de Contenido de URL**
---

En este ejemplo, obtenemos la información de Wikipedia (pero podria ser cualquier URL) sin usar su API. En su lugar accedemos al contenido de la pagina con Requests y se lo pasamos al modelo.
En resultado lo mostramos en pantalla y lo guardamos en un fichero.

In [ ]:
import requests
from langchain.schema.output_parser import StrOutputParser

# Función para obtener el contenido de una URL
def fetch_url_content(url):
    response = requests.get(url)
    return response.text

# LLM: Procesa el contenido de la URL
llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)

prompt_template = PromptTemplate(
    input_variables=["content"],
    template="Analiza el siguiente contenido y proporciona una síntesis:\n{content}"
)

# Cadena completa usando LCEL
chain = fetch_url_content | prompt_template | llm | StrOutputParser()

# Ejecutar la cadena y guardar el resultado
resumen = chain.invoke("https://es.wikipedia.org/wiki/Python")
with open("resumen.txt", "w") as file:
    file.write(resumen)
print("Resumen guardado en resumen.txt")
display(Markdown(resumen))

## **Ejemplo 10: ???**

No hay ejemplo 10. Planteate este reto:

XXXXXXXX


In [ ]:
# Escribe aqui la solución

## Solución

In [ ]:
# AQui va la solución

Siguiente tema / cuaderno:  

Curso de Langchain en Github: https://github.com/juanfranbrv/curso-langchain

Enlaces de interes sobre cadenas LCEL:

1. https://python.langchain.com/docs/concepts/lcel/
1. https://www.paradigmadigital.com/dev/uso-de-cadenas-langchain-gen-ai/

2. https://python.langchain.com/v0.1/docs/modules/chains/

3. https://medium.com/@anuragmishra_27746/practical-hands-on-with-langchain-expression-language-lcel-for-building-langchain-agent-chain-2a9364dc4ca3

4. https://medium.com/@itsmybestview/unbridling-the-power-of-langchain-framework-with-lcel-9e5f7bf8af74

5. https://www.youtube.com/playlist?list=PLGPnu4k-KSC1s7apArPZz1AavDJ4Mvitd

6. https://www.youtube.com/watch?v=LzxSY7197ns&t=12s

7. https://www.youtube.com/watch?v=H8DrR2pXbww&list=PLilZ1IiRt0R26etAQeQ9xa5h5UzK18zie&index=3

8. https://www.youtube.com/watch?v=8BV9TW490nQ&t=523s

9. https://www.youtube.com/watch?v=8BV9TW490nQ
